# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

In [1]:
from nltk.tokenize import sent_tokenize
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
from collections import Counter

In [2]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [3]:
import codecs
news = codecs.open( "lenta.txt", "r", "utf_8_sig" ).read() 

norm_news = normalize(news[:5000000])

vocab_news = Counter(norm_news)

probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

In [4]:
sentences_news = [['<start>','<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [5]:
unigrams_news = Counter()
bigrams_news = Counter()
threegrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence,2))
    threegrams_news.update(ngrammer(sentence))

In [6]:
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [197]:
matrix_news = lil_matrix((len(bigrams_news), 
                        len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bi_news = list(bigrams_news)
bi2id_news = {bi:i for i, bi in enumerate(id2bi_news)}


for ngram in threegrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + " " + word2
    matrix_news[bi2id_news[bigram], word2id_news[word3]] =  (threegrams_news[ngram]/
                                                                     bigrams_news[bigram])
    
matrix_news = csc_matrix(matrix_news)

In [198]:
import numpy as np

def apply_temperature(probas, temperature):
    log_probas = np.log(np.maximum(probas, 1e-10))  
    adjusted_log_probas = log_probas / temperature
    exp_probas = np.exp(adjusted_log_probas)
    adjusted_probabilities = exp_probas / np.sum(exp_probas)
    return adjusted_probabilities

In [232]:
def generate(matrix, id2word, bi2id, n=200, start='<start> <start>'):
    text = ["<start>" , "<start>"]
    current_idx = bi2id[start]
    for i in range(n):
        
        chosen_idx = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        chosen = id2word[chosen_idx]
        text.append(chosen)

        if chosen == '<end>':
            current_idx = bi2id['<start> <start>']
            text.extend(['<start>', '<start>'])
        else:
            current_idx = bi2id[" ".join(text[-2:])]
    return ' '.join(text)

In [365]:
print(generate(matrix_news, id2word_news, bi2id_news).replace('<end>', '\n').replace('<start> <start>', ''))

 периодически бои вспыхивают на улицах столицы красовались плакаты в поддержку мэра москвы 
  это вызвано давлением со стороны старопромысловского старосунженского районов и самых дальних деревень 
  вешняков так и не научились заключил генерал лебедь 
  высланные из столицы люди не могли найти англичане при обыске банка следствие обнаружило свидетельства позволяющие подозревать банк фламинго в незаконных финансовых операциях с использованием в россии финансовых средств на банковских корсчетах продолжают оставаться на своем заседании 24 января ставку рефинансирования на 0,5 процента 
  30 декабря 1999 года на святой земле первый президент россии борис ельцин намерен посетить ботлихский район дагестана а также увеличения числа работающих в области безопасности проведение совместных военных маневров и консультаций на высоком уровне боевой готовности и отвечают самым современным требованиям 
  однако слушания были посвященытеме коррупции и отмывания денег российскими преступниками через к

In [272]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

def compute_joint_proba_markov_assumption(text, word_counts, bigram_counts, threegram_counts):
    prob = 0
    tokens = normalize(text)

    for ngram in ngrammer(['<start>', '<start>'] + tokens + ['<end>']):
        word1, word2, word3 = ngram.split()
        bigram = word1 + " " + word2
        if bigram in bigram_counts and ngram in threegram_counts:
            prob += np.log(threegram_counts[ngram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5)

    return prob, len(tokens)

def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)
    
    return prob, len(tokens)

In [256]:
unigrams_news_perplex = Counter()
bigrams_news_perplex = Counter()
threegrams_news_perplex = Counter()

for sentence in sentences_news:
    unigrams_news_perplex.update(sentence)
    bigrams_news_perplex.update(ngrammer(sentence,2))
    threegrams_news_perplex.update(ngrammer(sentence))

In [257]:
norm_news_perplex = normalize(news[5000000:])

vocab_news_perplex = Counter(norm_news_perplex)

probas_news_perplex = Counter({word:c/len(norm_news_perplex) for word, c in vocab_news_perplex.items()})

In [296]:
import statistics
from statistics import fmean, median, mode

In [297]:
texts_for_perplex = generate(matrix_news, id2word_news, bi2id_news).replace('<end>', '\n').replace('<start> <start>', '').split('\n')

perplexes = []
for text in texts_for_perplex:
    perplexes.append(perplexity(*compute_joint_proba(text, probas_news_perplex)))

print(fmean(perplexes))
print(median(perplexes))
print(mode(perplexes))

4915.06976176523
4452.460986516732
4846.350957869431


## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [298]:
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score

In [360]:
def generate_with_beam_search(matrix, id2word, word2id, n=100, max_beams=5, start='<start> <start>'):

    initial_node = Beam(sequence=start.split(), score=np.log1p(0))
    beams = [initial_node]
    
    for i in range(n):
        new_beams = []

        for beam in beams:

            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue

            last_id = word2id[' '.join(beam.sequence[-2:])]

            probas = matrix[last_id].toarray()[0]

            top_idxs = probas.argsort()[:-(max_beams+1):-1]

            for top_id in top_idxs:
                if not probas[top_id]:
                    break
                
                new_sequence = beam.sequence + [id2word[top_id]]

                new_score = (beam.score + np.log1p(probas[top_id])) / len(new_sequence)
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)

        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]
    

    sorted_sequences = sorted(beams, key=lambda x: x.score, reverse=True)
    sorted_sequences = [" ".join(beam.sequence) for beam in sorted_sequences]
    return sorted_sequences

In [361]:
generation = generate_with_beam_search(matrix_news, id2word_news, bi2id_news)

res_gen = list()
for i in generation:
    n = len(i)
    res_gen.append(i[16:n-6])

from pprint import pprint
pprint(res_gen)

['как сообщает риа новости',
 'об этом риа новости',
 'об этом сообщает риа новости',
 'об этом сообщает агентство риа новости',
 'об этом сообщает итар-тасс со ссылкой на пресс-службу мчс россии сергей '
 'шойгу']


In [362]:
generation = generate_with_beam_search(matrix_news, id2word_news, bi2id_news, start = '<start> как')

res_gen = list()
for i in generation:
    n = len(i)
    res_gen.append(i[8:n-6])

from pprint import pprint
pprint(res_gen)

['как сообщили риа новости',
 'как сообщает риа новости',
 'как передает риа новости',
 'как сообщает агентство риа новости',
 'как сообщает итар-тасс со ссылкой на пресс-службу мчс россии сергей шойгу']


In [363]:
generation = generate_with_beam_search(matrix_news, id2word_news, bi2id_news, start = '<start> вчера')

res_gen = list()
for i in generation:
    n = len(i)
    res_gen.append(i[8:n-6])

from pprint import pprint
pprint(res_gen)

['вчера владельцы азс уже оказались закрыты',
 'вчера владельцы азс уже вели консультации с мэрией города',
 'вчера флойд прошел через багамы вырывая деревья срывая линии электропередач '
 'и крыши домов заливая морской водой улицы',
 'вчера владельцы азс уже вели консультации с представителями российского '
 'руководства',
 'вчера флойд прошел через багамы вырывая деревья срывая линии электропередач']


In [364]:
generation = generate_with_beam_search(matrix_news, id2word_news, bi2id_news, start = '<start> на протяжении')

res_gen = list()
for i in generation:
    n = len(i)
    res_gen.append(i[8:n-6])

from pprint import pprint
pprint(res_gen)

['на протяжении многих лет производители табачных изделий несовершеннолетним',
 'на протяжении многих лет импортировать в ливан эту продукцию',
 'на протяжении двух предыдущихнедель и повлияли на результаты выборов будут '
 'вне всякого сомнения оспорены теми кто будет ими недоволен',
 'на протяжении многих лет импортировать в ливан в связи с этим в одиночку',
 'на протяжении многих лет импортировать в ливан в связи с этим в одиночку '
 'подчеркнул виторино в интервью риа новости']
